In [1]:
import os
import numpy as np
import pandas as pd

from PIL import Image
from tqdm import tqdm

In [2]:
mask_model_ver = '095521'

DATA_DIR = "/root/autodl-tmp/cervical_spine/"
IMAGES_DIR = os.path.join(DATA_DIR, 'train_axial_images_jpeg95')
TARGET_DIR = os.path.join(DATA_DIR, f'train_axial_images_jpeg95_croped_{mask_model_ver}')

In [3]:
total_boundary_df = pd.read_csv(os.path.join(DATA_DIR, f'infered_boundary_{mask_model_ver}_2.csv'))
total_boundary_df['UID_Slice'] = total_boundary_df['StudyInstanceUID'] + '.' + total_boundary_df['Slice'].astype('string')
total_boundary_df = total_boundary_df.set_index('UID_Slice').astype({'Slice': int})
print(len(total_boundary_df))
total_boundary_df.head()

708779


,StudyInstanceUID,Slice,Start,pixel_spacing,xmin,ymin,xmax,ymax
UID_Slice,,,,,,,,
1.2.826.0.1.3680043.10001.0,1.2.826.0.1.3680043.10001,0,0,0.253906,54.076729,0.0,447.923271,393.846542
1.2.826.0.1.3680043.10001.1,1.2.826.0.1.3680043.10001,1,0,0.253906,0.000000,0.0,512.000000,512.000000
1.2.826.0.1.3680043.10001.2,1.2.826.0.1.3680043.10001,2,0,0.253906,0.000000,0.0,512.000000,512.000000
1.2.826.0.1.3680043.10001.3,1.2.826.0.1.3680043.10001,3,0,0.253906,0.000000,0.0,512.000000,512.000000
1.2.826.0.1.3680043.10001.4,1.2.826.0.1.3680043.10001,4,0,0.253906,0.000000,0.0,512.000000,512.000000


In [4]:
total_boundary_df = total_boundary_df.sort_values(['StudyInstanceUID', 'Slice']).reset_index()
total_boundary_df.head()

,UID_Slice,StudyInstanceUID,Slice,Start,pixel_spacing,xmin,ymin,xmax,ymax
0,1.2.826.0.1.3680043.10001.0,1.2.826.0.1.3680043.10001,0,0,0.253906,54.076729,0.0,447.923271,393.846542
1,1.2.826.0.1.3680043.10001.1,1.2.826.0.1.3680043.10001,1,0,0.253906,0.000000,0.0,512.000000,512.000000
2,1.2.826.0.1.3680043.10001.2,1.2.826.0.1.3680043.10001,2,0,0.253906,0.000000,0.0,512.000000,512.000000
3,1.2.826.0.1.3680043.10001.3,1.2.826.0.1.3680043.10001,3,0,0.253906,0.000000,0.0,512.000000,512.000000
4,1.2.826.0.1.3680043.10001.4,1.2.826.0.1.3680043.10001,4,0,0.253906,0.000000,0.0,512.000000,512.000000


In [5]:
def read_img(UID, Slice):
    return Image.open(os.path.join(IMAGES_DIR, UID, f'{int(Slice)}.jpeg'))

def crop_img(img, s):
    return img.crop((s.xmin, s.ymin, s.xmax, s.ymax)) 

def save_img(img, UID, Slice):
    target_dir = os.path.join(TARGET_DIR, UID)
    if os.path.exists(target_dir) is False:
        os.mkdir(target_dir)
    
    img.save(os.path.join(target_dir, f'{int(Slice)}.jpeg'))

r = read_img(total_boundary_df.iloc[0].StudyInstanceUID, total_boundary_df.iloc[0].Slice)
g = read_img(total_boundary_df.iloc[1].StudyInstanceUID, total_boundary_df.iloc[1].Slice)
                                                           
                                                           
for i in tqdm(range(len(total_boundary_df))):
    s = total_boundary_df.iloc[i]
    UID, Slice = s.StudyInstanceUID, s.Slice
    b = read_img(UID, Slice)
    
    if Slice == 0:
        r, g = b, b
    
    img = Image.merge('RGB', (r, g, b))
    img = crop_img(img, s)
    # crop
    save_img(img, UID, Slice)
    
    r = g
    g = b
    

100%|██████████| 708779/708779 [1:15:01<00:00, 157.45it/s]
